# Setup
(No need to read)

In [ ]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-oftgun1t
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-oftgun1t
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 218ebd6f491f47f5e2f64e4c4327548b60a093eb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 114.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━


## Installing the NodeSource Node.js 16.x repo...


## Populating apt-get cache...

+ apt-get update
Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,084 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,372 kB]
Hit:12 http://ppa.launchpad.net/g

In [ ]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [ ]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [ ]:
import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

In [ ]:
model = HookedTransformer.from_pretrained("gpt2-small")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


# Fibonacci

In [ ]:
example_prompt = "0, 1, 1, 2, 3, 5,"
example_answer = " 8"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '0', ',', ' 1', ',', ' 1', ',', ' 2', ',', ' 3', ',', ' 5', ',']
Tokenized answer: [' 8']


Performance on answer token:
Rank: 3        Logit: 16.12 Prob:  8.50% Token: | 8|

Top 0th token. Logit: 17.46 Prob: 32.54% Token: | 6|
Top 1th token. Logit: 17.32 Prob: 28.18% Token: | 7|
Top 2th token. Logit: 16.31 Prob: 10.30% Token: | 10|
Top 3th token. Logit: 16.12 Prob:  8.50% Token: | 8|
Top 4th token. Logit: 15.59 Prob:  4.99% Token: | 9|
Top 5th token. Logit: 14.98 Prob:  2.73% Token: | 11|
Top 6th token. Logit: 14.63 Prob:  1.91% Token: | 5|
Top 7th token. Logit: 14.53 Prob:  1.72% Token: | 12|
Top 8th token. Logit: 14.33 Prob:  1.41% Token: | 1|
Top 9th token. Logit: 13.49 Prob:  0.61% Token: | 2|


Ranks of the answer tokens: [(' 8', 3)]

In [ ]:
example_prompt = "0 1 1 2 3 5"
example_answer = " 8"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '0', ' 1', ' 1', ' 2', ' 3', ' 5']
Tokenized answer: [' 8']


Performance on answer token:
Rank: 3        Logit: 14.84 Prob:  2.82% Token: | 8|

Top 0th token. Logit: 18.06 Prob: 70.52% Token: | 6|
Top 1th token. Logit: 15.19 Prob:  4.01% Token: |
|
Top 2th token. Logit: 15.06 Prob:  3.53% Token: | 0|
Top 3th token. Logit: 14.84 Prob:  2.82% Token: | 8|
Top 4th token. Logit: 14.80 Prob:  2.71% Token: | 10|
Top 5th token. Logit: 14.66 Prob:  2.35% Token: | 7|
Top 6th token. Logit: 14.49 Prob:  1.99% Token: | 5|
Top 7th token. Logit: 14.22 Prob:  1.52% Token: | 4|
Top 8th token. Logit: 13.74 Prob:  0.94% Token: | 9|
Top 9th token. Logit: 13.38 Prob:  0.66% Token: | 1|


Ranks of the answer tokens: [(' 8', 3)]

In [ ]:
example_prompt = "0 1 1 2 3 5 8 13 21"
example_answer = " 34"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '0', ' 1', ' 1', ' 2', ' 3', ' 5', ' 8', ' 13', ' 21']
Tokenized answer: [' 34']


Performance on answer token:
Rank: 17       Logit: 16.04 Prob:  1.55% Token: | 34|

Top 0th token. Logit: 17.60 Prob:  7.37% Token: | 22|
Top 1th token. Logit: 17.58 Prob:  7.26% Token: | 30|
Top 2th token. Logit: 17.43 Prob:  6.25% Token: | 25|
Top 3th token. Logit: 17.39 Prob:  5.97% Token: | 24|
Top 4th token. Logit: 17.31 Prob:  5.52% Token: |
|
Top 5th token. Logit: 17.12 Prob:  4.58% Token: | 32|
Top 6th token. Logit: 17.09 Prob:  4.44% Token: | 23|
Top 7th token. Logit: 16.99 Prob:  4.00% Token: | 20|
Top 8th token. Logit: 16.87 Prob:  3.56% Token: | 29|
Top 9th token. Logit: 16.72 Prob:  3.07% Token: | 21|


Ranks of the answer tokens: [(' 34', 17)]

# 2 4 6 8

In [ ]:
example_prompt = "2 4 6 8 10 12"
example_answer = " 14"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

NameError: ignored